# 런타임에 체인 내부 구성하기

이번 튜토리얼은 Chain 을 호출시 다양한 옵션을 동적으로 설정할 수 있는 방법을 알아보겠습니다.

다음의 2가지 방식으로 동적 구성을 할 수 있습니다.

- 첫째, `configurable_fields` 메서드입니다. <br>이 메서드를 통해 실행 가능한 객체의 특정 필드를 구성할 수 있습니다.

- 둘째, `configurable_alternatives` 메서드입니다. <br>이 메서드를 사용하면 런타임 중에 설정할 수 있는 특정 실행 가능한 객체에 대한 대안을 나열할 수 있습니다.


## configurable_fields

`configurable_fields` 는 시스템의 설정 값을 정의하는 필드를 의미합니다.

## 동적 속성 지정

`ChatOpenAI` 을 사용할 때, 우리는 `model_name` 와 같은 설정을 조정할 수 있습니다.

`model_name` 은 GPT 의 버전을 명시할 때 사용하는 속성입니다. <br>예를 들어, `gpt-4o`, `gpt-4o-mini` 등을 설정하여 모델을 선택할 수 있습니다.

만약, 고정된 `model_name` 이 아닌 동적으로 모델을 지정하고 싶을 때는 <br>다음과 같이 `ConfigurableField` 를 활용하여 동적으로 설정할 수 있는 속성 값으로 변환할 수 있습니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH13-LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH13-LCEL-Advanced


`configurable_fields` 메서드를 사용하여 `model_name` 속성을 동적 구성 가능한 필드로 지정합니다.


In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

model.invoke("대한민국의 수도는 어디야?").__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 9,
   'prompt_tokens': 15,
   'total_tokens': 24,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-2024-08-06',
  'system_fingerprint': 'fp_703d4ff298',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-d5c04e77-84fe-4403-8f77-09ed6784c214-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 9,
  'total_tokens': 24,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [4]:
model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(  # model_name 은 원래 ChatOpenAI 의 필드입니다.
        id="gpt_version",  # model_name 의 id 를 설정합니다.
        name="Version of GPT",  # model_name 의 이름을 설정합니다. #크게 중요하지 않음
        # model_name 의 설명을 설정합니다.
        description="Official model name of GPTs. ex) gpt-4o, gpt-4o-mini",  # 크게 중요하지 않음
    )
)

`model.invoke()` 호출시 `config={"configurable": {"키": "값"}}` 형식으로 동적 지정할 수 있습니다.


In [5]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-3.5-turbo 로 설정합니다.
    # 실행 전에 model_name을 변경하게됨
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
).__dict__

{'content': '대한민국의 수도는 서울이야.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 17,
   'prompt_tokens': 22,
   'total_tokens': 39,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-733f16f9-c432-48cb-92b6-236dde7ec6fc-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 22,
  'output_tokens': 17,
  'total_tokens': 39,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

이번에는 `gpt-4o-mini` 모델을 사용해보겠습니다. 출력에 바뀐 모델을 확인하세요.


In [6]:
model.invoke(
    # gpt_version 을 gpt-4o-mini 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4o-mini"}},
).__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 9,
   'prompt_tokens': 15,
   'total_tokens': 24,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_01aeff40ea',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-f2ea9dfd-ae87-43c8-b404-1e2d2b628d27-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 9,
  'total_tokens': 24,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

`model` 객체의 `with_config()` 메서드를 사용하여 `configurable` 매개변수를 설정할 수도 있습니다. 이전과 동작하는 방식은 동일합니다.


In [7]:
model.with_config(configurable={"gpt_version": "gpt-4o-mini"}).invoke(
    "대한민국의 수도는 어디야?"
).__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 9,
   'prompt_tokens': 15,
   'total_tokens': 24,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_01aeff40ea',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-07c696b0-9a68-4930-b3bb-346d320fd2d0-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 9,
  'total_tokens': 24,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

또한 이 함수를 체인의 일부로 사용할 때에도 동일한 방식으로 활용할 수 있습니다.


In [8]:
# 템플릿에서 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{x} 보다 큰 위의 난수를 선택합니다.")
chain = (
    prompt | model
)  # 프롬프트와 모델을 연결하여 체인을 생성합니다. 프롬프트의 출력이 모델의 입력으로 전달됩니다.

In [ ]:
chain.invoke(
    {"x": 0},
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
).__dict__  # 체인을 호출하고 입력 변수 "x"에 0을 전달합니다.

{'content': '5',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 2,
   'prompt_tokens': 21,
   'total_tokens': 23,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-ea1b600e-ea89-4f12-b741-2d5e92afc08e-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 21,
  'output_tokens': 2,
  'total_tokens': 23,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [11]:
# 체인을 호출할 때 설정을 지정하여 체인을 호출할 수 있습니다.
chain.with_config(configurable={"gpt_version": "gpt-4o"}).invoke({"x": 0}).__dict__

{'content': '위의 요청은 0보다 큰 난수를 생성하라는 의미로 이해됩니다. 난수를 생성하기 위해서는 특정 프로그래밍 언어나 도구가 필요합니다. 예를 들어, 파이썬에서는 `random` 모듈을 사용하여 난수를 생성할 수 있습니다. 다음은 파이썬에서 0보다 큰 난수를 생성하는 예시입니다.\n\n```python\nimport random\n\n# 0보다 큰 난수를 생성합니다.\nrandom_number = random.uniform(0.0001, 1.0)\nprint(random_number)\n```\n\n이 코드는 0과 1 사이의 실수 난수를 생성합니다. 필요에 따라 범위를 조정할 수 있습니다. 다른 프로그래밍 언어에서도 유사한 기능을 제공하는 라이브러리를 사용할 수 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 169,
   'prompt_tokens': 17,
   'total_tokens': 186,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-2024-08-06',
  'system_fingerprint': 'fp_b7d65f1a5b',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-d5835d5d-800b-4ce0-955e-9f8ff11062a6-0',
 'example': False,
 't

In [12]:
chain.invoke({"x": 0})

AIMessage(content='5', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 21, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6ea6138f-d6cd-4d40-b204-8ee2d2247b55-0', usage_metadata={'input_tokens': 21, 'output_tokens': 2, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## HubRunnable: LangChain Hub의 설정 변경

`HubRunnable` 을 사용하면 Hub 에 등록된 프롬프트의 전환을 용이하게 합니다.


In [13]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("teddynote/rag-prompt-korean").configurable_fields(
    # 소유자 저장소 커밋을 설정하는 ConfigurableField
    # teddynote/rag-prompt-korean 를 변경하겠다.
    owner_repo_commit=ConfigurableField(
        # 필드의 ID
        id="hub_commit",
        # 필드의 이름
        name="Hub Commit",
        # 필드에 대한 설명
        description="Korean RAG prompt by teddynote",
    )
)
prompt
# RunnableConfigurableFields(default=HubRunnable(bound=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'rag-prompt-korean', 'lc_hub_commit_hash': '4214de29c72c82d13121c710b0ee7021d31ad088f0eb1d6ddebcaaeab46b1f8b'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='#Question: \n{question} \n\n#Context: \n{context} \n\n#Answer:'), additional_kwargs={})]), kwargs={}, config={}, config_factories=[],
# owner_repo_commit='teddynote/rag-prompt-korean'), fields={'owner_repo_commit': ConfigurableField(id='hub_commit', name='Hub Commit', description='Korean RAG prompt by teddynote', annotation=None, is_shared=False)})

RunnableConfigurableFields(default=HubRunnable(bound=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'rag-prompt-korean', 'lc_hub_commit_hash': '4214de29c72c82d13121c710b0ee7021d31ad088f0eb1d6ddebcaaeab46b1f8b'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='#Question: \n{questi

별도의 `with_config` 지정 없이 `prompt.invoke()` 메서드를 호출하면 처음 설정한 `"rlm/rag-prompt"` hub 에 등록된 프롬프트를 pull 하여 가져옵니다.


In [14]:
# prompt 객체의 invoke 메서드를 호출하여 "question"과 "context" 매개변수를 전달합니다.
prompt.invoke({"question": "Hello", "context": "World"}).messages

[SystemMessage(content="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='#Question: \nHello \n\n#Context: \nWorld \n\n#Answer:', additional_kwargs={}, response_metadata={})]

In [15]:
prompt.with_config(
    # hub_commit 을 teddynote/simple-summary-korean 으로 설정합니다.
    configurable={"hub_commit": "teddynote/simple-summary-korean"}
).invoke({"context": "Hello"})

ChatPromptValue(messages=[HumanMessage(content='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: Hello\n\nSUMMARY:', additional_kwargs={}, response_metadata={})])

## Configurable Alternatives: Runnable 객체 자체의 대안 설정

런타임에 설정할 수 있는 Runnable 에 대한 대안을 구성합니다.

**구성 가능한 대안들**

`ChatAnthropic` 의 구성 가능한 언어 모델은 다양한 작업과 컨텍스트에 적용할 수 있는 유연성을 제공합니다.

동적으로 설정(Config) 값을 변경하기 위하여 모델에 설정하는 파라미터를 ConfigurableField 객체로 설정합니다.

- `model`: 사용할 기본 언어 모델을 지정합니다.

- `temperature`: 0에서 1 사이의 값으로, 샘플링의 무작위성을 제어합니다. <br>값이 낮을수록 더 결정적이고 반복적인 출력이 생성되며, 값이 높을수록 더 다양하고 창의적인 출력이 생성됩니다.

### LLM 객체의 대안(alternatives) 설정 방법

LLM(Large Language Model)을 활용하여 이를 수행하는 방법을 살펴보겠습니다.

[참고]

- `ChatAnthropic` 모델을 사용하기 위하여 API KEY를 발급받아 설정해야합니다.
- 링크: https://console.anthropic.com/dashboard
- 아래 주석을 해제하고 API KEY를 설정하거나, `.env` 파일에 설정합니다.
  
`ANTHROPIC_API_KEY` 환경변수를 설정합니다.

In [ ]:
# import os

# os.environ["ANTHROPIC_API_KEY"] = "ANTHROPIC API KEY를 입력합니다."

In [16]:
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(
    temperature=0, model="claude-3-5-sonnet-20240620"
).configurable_alternatives(  # 대안을 제공해줌
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(model="gpt-4o-mini"),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4o=ChatOpenAI(model="gpt-4o"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm

`chain.invoke()` 메서드를 기본 LLM 인 `ChatAnthropic` 을 활용한 체인을 호출합니다.


In [17]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "뉴진스"}).__dict__

{'content': '뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸그룹입니다. 주요 특징은 다음과 같습니다:\n\n1. 소속사: ADOR (HYBE Labels의 자회사)\n\n2. 멤버: 민지, 하니, 다니엘, 해린, 혜인\n\n3. 데뷔곡: "Attention", "Hype Boy", "Cookie"\n\n4. 특징:\n   - 10대 멤버들로 구성된 신선한 이미지\n   - Y2K, 레트로 감성을 현대적으로 재해석한 콘셉트\n   - 독특한 마케팅 전략 (사전 홍보 없이 갑작스러운 데뷔)\n\n5. 성과:\n   - 데뷔 앨범 \'New Jeans\'로 빌보드 200 차트 진입\n   - 여러 음원 차트 1위 기록\n   - 각종 신인상 수상\n\n뉴진스는 독특한 음악 스타일과 콘셉트로 K-pop 시장에 새로운 바람을 일으키며 빠르게 인기를 얻고 있는 그룹입니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01UYs3uaTfBmbK19dfNRRwNT',
  'model': 'claude-3-5-sonnet-20240620',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 30,
   'output_tokens': 389}},
 'type': 'ai',
 'name': None,
 'id': 'run-0015ead0-2125-4b5b-be1d-64e26529354f-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 30,
  'output_tokens': 389,
  'total_tokens': 4

`chain.with_config(configurable={"llm": "모델"})`를 사용하여 사용할 `llm`으로 다른 모델을 지정할 수 있습니다.


In [18]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "뉴진스"}).__dict__

{'content': "뉴진스(NewJeans)는 2022년 데뷔한 대한민국의 걸 그룹으로, 하이브의 자회사인 어도어(ADOR) 소속입니다. 그룹의 이름은 '새로운 청바지'라는 의미를 담고 있으며, 이는 새로운 트렌드와 스타일을 제시하고자 하는 의지를 나타냅니다. 뉴진스는 독특한 음악 스타일과 신선한 컨셉으로 주목받아 왔으며, 특히 'Attention', 'Hype Boy', 'Cookie' 등의 곡으로 큰 인기를 얻었습니다. 멤버들은 개별적으로도 다양한 매력을 발산하며, 그룹의 비주얼과 퍼포먼스 모두에서 높은 평가를 받고 있습니다. 그들의 음악은 K-pop의 다양한 요소를 접목하여 많은 사랑을 받고 있습니다.",
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 175,
   'prompt_tokens': 18,
   'total_tokens': 193,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_01aeff40ea',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-ec8572b0-860f-4ccf-ba80-864e176beb6b-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls':

체인의 설정을 변경하여 사용할 언어 모델을 `gpt4o` 로 지정합니다.


In [ ]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "gpt4o"}).invoke({"topic": "뉴진스"}).__dict__

체인의 설정을 변경하여 사용할 언어 모델을 `anthropic` 로 지정합니다.


In [19]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "anthropic"}).invoke(
    {"topic": "뉴진스"}
).__dict__

{'content': '뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸그룹입니다. 주요 특징은 다음과 같습니다:\n\n1. 소속사: ADOR (HYBE Labels의 자회사)\n\n2. 멤버: 민지, 하니, 다니엘, 해린, 혜인\n\n3. 데뷔곡: "Attention", "Hype Boy", "Cookie"\n\n4. 특징:\n   - Y2K, 90년대 감성을 현대적으로 재해석한 콘셉트\n   - 자연스럽고 편안한 이미지 추구\n   - 뛰어난 보컬과 퍼포먼스 능력\n\n5. 주요 성과:\n   - 데뷔 앨범으로 빌보드 200 차트 진입\n   - 여러 음악 방송 1위 달성\n   - 각종 신인상 수상\n\n6. 인기 요인:\n   - 독특한 마케팅 전략\n   - 멤버들의 다국적 배경 (한국, 호주, 베트남 등)\n   - 신선하고 차별화된 음악 스타일\n\n뉴진스는 데뷔 이후 빠르게 인기를 얻으며 4세대 K-pop을 대표하는 그룹으로 자리매김하고 있습니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01H7awvjDVwN2akqtLPbnvrD',
  'model': 'claude-3-5-sonnet-20240620',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 30,
   'output_tokens': 437}},
 'type': 'ai',
 'name': None,
 'id': 'run-93cf638e-6ca6-4cb6-a13e-67de0d9350a5-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens

## 프롬프트의 대안 설정 방법

프롬프트도 이전의 LLM 대안 설정 방법과 유사한 작업을 수행할 수 있습니다.


In [20]:
# 언어 모델을 초기화하고 temperature를 0으로 설정합니다.
llm = ChatOpenAI(temperature=0)

prompt = PromptTemplate.from_template(
    "{country} 의 수도는 어디야?"  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="capital",
    # 'area'이라는 새로운 옵션을 추가합니다.
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    # 'population'이라는 새로운 옵션을 추가합니다.
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    # 'eng'이라는 새로운 옵션을 추가합니다.
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요."),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

# 프롬프트와 언어 모델을 연결하여 체인을 생성합니다.
chain = prompt | llm

아무런 설정 변경이 없다면 기본 프롬프트가 입력됩니다.


In [21]:
# config 변경 없이 체인을 호출합니다.
chain.invoke({"country": "대한민국"})

AIMessage(content='대한민국의 수도는 서울이다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 23, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d55bb061-defa-4ea9-98a6-45eb765e1a90-0', usage_metadata={'input_tokens': 23, 'output_tokens': 17, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

`with_config` 로 다른 프롬프트를 호출합니다.


In [22]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "area"}).invoke({"country": "대한민국"})

AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 25, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f3b79100-0876-4837-acb2-352525f74264-0', usage_metadata={'input_tokens': 25, 'output_tokens': 26, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "population"}).invoke({"country": "대한민국"})

AIMessage(content='2021년 9월 기준 대한민국의 인구는 약 5천 130만 명입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 24, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-371b0d52-e4eb-4088-ae69-9d5456487fa5-0', usage_metadata={'input_tokens': 24, 'output_tokens': 35, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

이번에는 `eng` 프롬프트를 사용하여 번역을 요청합니다. 이때 전달할 입력 변수는 `input`입니다.


In [24]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "eng"}).invoke({"input": "사과는 맛있어!"})

AIMessage(content='Apples are delicious!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 28, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c34bc229-2b95-4de9-8f09-10e93856247b-0', usage_metadata={'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 프롬프트 & LLM 모두 변경

프롬프트와 LLM을 사용하여 여러 가지 사항을 구성할 수 있습니다.

다음은 프롬프트와 LLM을 모두 사용하여 이를 수행하는 예시입니다.


In [25]:
llm = ChatAnthropic(
    temperature=0, model="claude-3-5-sonnet-20240620"
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4o-mini")`와 동일합니다.
    openai=ChatOpenAI(model="gpt-4o-mini"),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4o")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4o"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

prompt = PromptTemplate.from_template(
    "{company} 에 대해서 20자 이내로 설명해 줘."  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="description",
    # 'founder'이라는 새로운 옵션을 추가합니다.
    founder=PromptTemplate.from_template("{company} 의 창립자는 누구인가요?"),
    # 'competitor'이라는 새로운 옵션을 추가합니다.
    competitor=PromptTemplate.from_template("{company} 의 경쟁사는 누구인가요?"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
chain = prompt | llm

In [26]:
# with_config 로 설정 값을 지정하여 구성할 수 있습니다.
chain.with_config(configurable={"prompt": "founder", "llm": "openai"}).invoke(
    # 사용자가 제공한 회사에 대한 처리를 요청합니다.
    {"company": "애플"}
).__dict__

{'content': '애플의 창립자는 스티브 잡스(Steve Jobs), 스티브 워즈니악(Steve Wozniak), 그리고 로널드 웨인(Ronald Wayne)입니다. 이들은 1976년에 애플 컴퓨터 회사를 설립했습니다. 스티브 잡스와 스티브 워즈니악은 주로 제품 개발과 경영을 맡았고, 로널드 웨인은 회사 설립 초기 몇 주 동안 함께했지만, 곧 회사를 떠났습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 111,
   'prompt_tokens': 17,
   'total_tokens': 128,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_d02d531b47',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-aef3fc47-5dd5-4a18-93c7-6f896dee4986-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 17,
  'output_tokens': 111,
  'total_tokens': 128,
  'input_token_details': {'audio': 0, 'cache_re

In [27]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"llm": "anthropic"}).invoke(
    {"company": "애플"}
).__dict__

{'content': '애플은 혁신적인 기술 제품을 만드는 세계적인 IT 기업입니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_018eGJBHaek8BoWDK8YQhmSk',
  'model': 'claude-3-5-sonnet-20240620',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 31,
   'output_tokens': 38}},
 'type': 'ai',
 'name': None,
 'id': 'run-5110c553-164b-423d-a0a8-cef5b5517b21-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 31,
  'output_tokens': 38,
  'total_tokens': 69,
  'input_token_details': {'cache_read': 0, 'cache_creation': 0}}}

In [28]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"prompt": "competitor"}).invoke(
    {"company": "애플"}
).__dict__

{'content': '애플의 주요 경쟁사들은 다음과 같습니다:\n\n1. 삼성전자: 스마트폰, 태블릿, 웨어러블 기기 등에서 경쟁\n\n2. 구글: 모바일 운영체제(안드로이드), 클라우드 서비스, 스마트홈 기기 등\n\n3. 마이크로소프트: 컴퓨터 운영체제, 클라우드 서비스, 태블릿 등\n\n4. 아마존: 클라우드 서비스, 스마트홈 기기, 디지털 콘텐츠 등\n\n5. 화웨이: 스마트폰, 통신장비 등 (일부 국가에서 제한적)\n\n6. 소니: 헤드폰, 스마트폰, 게임 콘솔 등\n\n7. 델, HP, 레노버: 개인용 컴퓨터 시장\n\n8. 스포티파이, 넷플릭스: 스트리밍 서비스\n\n9. 페이스북(메타): AR/VR 기술, 메타버스\n\n10. 테슬라: 자율주행 기술, 전기차 (애플의 자동차 프로젝트와 관련)\n\n이 회사들은 다양한 제품과 서비스 영역에서 애플과 경쟁하고 있습니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01FdBooasZ1T48xsLupqQJtt',
  'model': 'claude-3-5-sonnet-20240620',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 26,
   'output_tokens': 427}},
 'type': 'ai',
 'name': None,
 'id': 'run-6e0a8196-a5fd-4a94-90e9-7a0f8ebbbc51-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 26,
  'output_tokens': 427,
  'total_tokens': 453,
  'input_

In [29]:
# 하나만 구성하려는 경우
chain.invoke({"company": "애플"}).__dict__

{'content': '애플은 혁신적인 기술 제품을 만드는 세계적인 IT 기업입니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01L7mK57YHDkbwHbbYbKzBQQ',
  'model': 'claude-3-5-sonnet-20240620',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 31,
   'output_tokens': 38}},
 'type': 'ai',
 'name': None,
 'id': 'run-e2e0ea2e-a96e-4fbd-b21f-c15fc997e22f-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 31,
  'output_tokens': 38,
  'total_tokens': 69,
  'input_token_details': {'cache_read': 0, 'cache_creation': 0}}}

## 설정 저장

구성된 체인을 별도의 객체로 쉽게 저장할 수 있습니다. <br>예를 들어, 특정 작업을 위해 사용자 정의된 체인을 구성한 후, <br>이를 재사용 가능한 객체로 저장함으로써 향후 유사한 작업에서 손쉽게 활용할 수 있습니다.


In [30]:
# with_config 로 설정을 변경하여 생성한 체인을 별도의 변수에 저장합니다.
gpt4_competitor_chain = chain.with_config(
    configurable={"llm": "gpt4", "prompt": "competitor"}
)

In [31]:
# 체인을 호출합니다.
gpt4_competitor_chain.invoke({"company": "애플"}).__dict__

{'content': '애플의 주요 경쟁사는 여러 기술 및 전자 제품 분야에서 다양합니다. 몇 가지 주요 경쟁사를 살펴보면:\n\n1. **삼성전자**: 스마트폰, 태블릿, 웨어러블 기기 등 다양한 제품군에서 애플과 경쟁하고 있습니다.\n2. **구글**: 안드로이드 운영체제를 통해 스마트폰 시장에서 경쟁하며, 픽셀 스마트폰 및 구글 홈과 같은 하드웨어 제품도 애플과 경쟁합니다.\n3. **마이크로소프트**: PC 운영체제 및 소프트웨어 분야에서 경쟁하며, 서피스 태블릿과 같은 하드웨어 제품도 있습니다.\n4. **아마존**: 아마존의 에코(알렉사) 디바이스는 스마트홈 및 AI 스피커 분야에서 애플의 홈팟과 경쟁합니다.\n5. **화웨이 및 샤오미**: 주로 스마트폰 및 기타 전자 기기 시장에서 애플과 경쟁하는 중국 기업들입니다.\n6. **페이스북(메타)**: VR 및 AR 분야에서 경쟁하며, 메타버스 개발에서도 경쟁 관계에 있습니다.\n\n이 외에도 다양한 기술 및 엔터테인먼트 서비스 분야에서 여러 기업들과 경쟁하고 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 272,
   'prompt_tokens': 16,
   'total_tokens': 288,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-2024-08-06',
  'system_fingerprint': 'fp_f785eb5f47',
  'finish_reas